# 🎯 Sistema de Geração Automática de Atas - UFS

**Demonstração Interativa para Conselho Universitário**

Este notebook implementa uma interface visual para demonstrar o sistema de geração automática de atas de reunião desenvolvido para a Universidade Federal de Sergipe (UFS).

## 🚀 Funcionalidades

- ✅ **Upload de áudio** - Suporte a .mp3, .wav, .m4a
- ✅ **Diarização automática** - Separação de speakers com IA
- ✅ **Transcrição precisa** - Conversão áudio → texto com Whisper
- ✅ **Geração de ata** - Criação automática de ata estruturada
- ✅ **Interface amigável** - Demonstração visual completa

---

## 📦 Instalação das Dependências

In [ ]:
# Instalação das dependências necessárias
!pip install gradio -q
!pip install git+https://github.com/openai/whisper.git -q
!pip install openai -q
!pip install pyannote.audio -q
!pip install torch torchvision torchaudio -q
!pip install pydub -q

In [ ]:
# Imports necessários
import gradio as gr
import whisper
import os
from openai import OpenAI
from pyannote.audio import Pipeline
import torch
from datetime import datetime
import json
import tempfile
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")

## ⚙️ Configuração dos Modelos de IA

In [ ]:
# Configuração da API OpenAI
# IMPORTANTE: Substitua pela sua chave de API
OPENAI_API_KEY = "your-openai-api-key-here"  # ou use os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY == "your-openai-api-key-here":
    print("⚠️ ATENÇÃO: Configure sua chave da OpenAI API")
    print("   Edite a variável OPENAI_API_KEY acima")
else:
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("✅ Cliente OpenAI configurado!")

In [ ]:
# Carregamento dos modelos
print("🔄 Carregando modelo Whisper...")
whisper_model = whisper.load_model("small")
print("✅ Whisper carregado!")

print("🔄 Configurando pipeline de diarização...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"   Dispositivo: {device}")

# Nota: Para usar pyannote, aceite os termos em:
# https://huggingface.co/pyannote/speaker-diarization
# https://huggingface.co/pyannote/segmentation
try:
    diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1")
    diarization_pipeline.to(device)
    print("✅ Pipeline de diarização configurado!")
    DIARIZATION_AVAILABLE = True
except Exception as e:
    print(f"⚠️ Diarização não disponível: {e}")
    print("   Sistema funcionará sem separação de speakers")
    DIARIZATION_AVAILABLE = False

## 🛠️ Funções do Sistema

In [ ]:
def perform_diarization(audio_path):
    """
    Realiza diarização (separação de speakers) do áudio
    """
    if not DIARIZATION_AVAILABLE:
        return []
    
    try:
        diarization = diarization_pipeline(audio_path)
        speakers_info = []
        
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            speakers_info.append({
                "speaker": speaker,
                "start": turn.start,
                "end": turn.end,
                "duration": turn.end - turn.start
            })
        
        return speakers_info
    except Exception as e:
        print(f"Erro na diarização: {e}")
        return []

In [ ]:
def transcribe_with_diarization(audio_path, speakers_info):
    """
    Transcreve áudio com informações de diarização
    """
    try:
        # Transcrição completa com Whisper
        result = whisper_model.transcribe(audio_path, language="pt")
        full_transcription = result["text"]
        segments = result.get("segments", [])
        
        if not speakers_info or not segments:
            return [], full_transcription
        
        # Associar segmentos com speakers
        speaker_transcriptions = []
        
        for segment in segments:
            segment_center = (segment["start"] + segment["end"]) / 2
            assigned_speaker = "PARTICIPANTE"
            
            # Encontrar speaker para este segmento
            for speaker_info in speakers_info:
                if speaker_info["start"] <= segment_center <= speaker_info["end"]:
                    assigned_speaker = speaker_info["speaker"]
                    break
            
            speaker_transcriptions.append({
                "speaker": assigned_speaker,
                "start": segment["start"],
                "end": segment["end"],
                "text": segment["text"].strip(),
                "duration": segment["end"] - segment["start"]
            })
        
        return speaker_transcriptions, full_transcription
    
    except Exception as e:
        print(f"Erro na transcrição: {e}")
        return [], ""

In [ ]:
def generate_speaker_stats(speaker_transcriptions):
    """
    Gera estatísticas dos participantes
    """
    speaker_stats = defaultdict(lambda: {"total_time": 0, "segments": 0, "words": 0})
    
    for segment in speaker_transcriptions:
        speaker = segment["speaker"]
        speaker_stats[speaker]["total_time"] += segment["duration"]
        speaker_stats[speaker]["segments"] += 1
        speaker_stats[speaker]["words"] += len(segment["text"].split())
    
    return dict(speaker_stats)

In [ ]:
def generate_meeting_minutes(transcription, speaker_stats=None):
    """
    Gera ata de reunião usando OpenAI
    """
    try:
        # Preparar contexto dos participantes
        speaker_context = ""
        if speaker_stats:
            speaker_context = "\n\n=== PARTICIPANTES IDENTIFICADOS ===\n"
            for speaker, stats in speaker_stats.items():
                speaker_context += f"- {speaker}: {stats['total_time']:.1f}s de fala, {stats['segments']} intervenções\n"
        
        system_prompt = """Você é um assistente especializado em gerar atas de reunião para o contexto universitário brasileiro.
        
        Sua tarefa é analisar a transcrição de uma reunião e criar uma ata formal e estruturada seguindo os padrões acadêmicos.
        
        A ata deve conter:
        1. CABEÇALHO - Data, participantes, tipo de reunião
        2. PAUTA - Principais tópicos discutidos
        3. DELIBERAÇÕES - Decisões tomadas e votações
        4. ENCAMINHAMENTOS - Ações futuras e responsáveis
        5. OBSERVAÇÕES - Informações adicionais relevantes
        
        Use linguagem formal, objetiva e organize as informações de forma clara e hierárquica.
        Identifique decisões importantes, pontos de consenso e discordância quando aplicável."""
        
        user_prompt = f"""TRANSCRIÇÃO DA REUNIÃO:
        {transcription}
        {speaker_context}
        
        Por favor, gere uma ata completa, formal e bem estruturada baseada nesta transcrição.
        Organize as informações de forma profissional adequada para o ambiente universitário."""
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2,
            max_tokens=3000
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        return f"Erro na geração da ata: {str(e)}"

In [ ]:
def process_audio_file(audio_file, progress=gr.Progress()):
    """
    Função principal que processa o arquivo de áudio
    """
    if audio_file is None:
        return "❌ Nenhum arquivo de áudio foi enviado.", "", "", ""
    
    try:
        progress(0, desc="🎵 Carregando arquivo de áudio...")
        
        # Etapa 1: Diarização
        progress(0.1, desc="🎭 Identificando participantes (diarização)...")
        speakers_info = perform_diarization(audio_file)
        
        num_speakers = len(set([s['speaker'] for s in speakers_info])) if speakers_info else 1
        
        # Etapa 2: Transcrição
        progress(0.4, desc="🎤 Transcrevendo áudio...")
        speaker_transcriptions, full_transcription = transcribe_with_diarization(audio_file, speakers_info)
        
        if not full_transcription:
            return "❌ Erro na transcrição do áudio.", "", "", ""
        
        # Etapa 3: Estatísticas
        progress(0.6, desc="📊 Calculando estatísticas...")
        speaker_stats = generate_speaker_stats(speaker_transcriptions)
        
        # Etapa 4: Geração da ata
        progress(0.8, desc="📝 Gerando ata de reunião...")
        meeting_minutes = generate_meeting_minutes(full_transcription, speaker_stats)
        
        progress(1.0, desc="✅ Processamento concluído!")
        
        # Formatação dos resultados
        stats_text = f"""## 📊 Estatísticas da Reunião

**Participantes identificados:** {num_speakers}
**Duração da transcrição:** {len(full_transcription)} caracteres
**Processado em:** {datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}

### Participação por Speaker:
"""
        
        if speaker_stats:
            for speaker, stats in sorted(speaker_stats.items()):
                stats_text += f"\n- **{speaker}**: {stats['total_time']:.1f}s ({stats['segments']} intervenções, {stats['words']} palavras)"
        else:
            stats_text += "\n- Não foi possível separar por participantes"
        
        # Transcrição formatada
        transcription_display = f"""## 🎤 Transcrição Completa

{full_transcription[:2000]}{'...' if len(full_transcription) > 2000 else ''}
"""
        
        # Ata formatada
        ata_display = f"""## 📋 Ata de Reunião Gerada

{meeting_minutes}
"""
        
        success_msg = f"""✅ **Processamento concluído com sucesso!**

🎯 **Arquivo processado:** {os.path.basename(audio_file)}
🕒 **Horário:** {datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}
👥 **Participantes:** {num_speakers} identificados
📄 **Transcrição:** {len(full_transcription)} caracteres
"""
        
        return success_msg, stats_text, transcription_display, ata_display
    
    except Exception as e:
        error_msg = f"❌ **Erro no processamento:** {str(e)}"
        return error_msg, "", "", ""

## 🎨 Interface Gradio

In [ ]:
def create_interface():
    """
    Cria a interface Gradio
    """
    
    # CSS personalizado
    css = """
    .gradio-container {
        max-width: 1200px !important;
    }
    .header {
        text-align: center;
        background: linear-gradient(90deg, #1f4e79, #2e7d32);
        color: white;
        padding: 20px;
        border-radius: 10px;
        margin-bottom: 20px;
    }
    .status-success {
        background-color: #e8f5e8;
        border: 1px solid #4caf50;
        border-radius: 5px;
        padding: 10px;
    }
    """
    
    with gr.Blocks(css=css, title="Sistema de Atas UFS") as interface:
        
        # Cabeçalho
        gr.HTML("""
        <div class="header">
            <h1>🎯 Sistema de Geração Automática de Atas</h1>
            <h2>Universidade Federal de Sergipe - UFS</h2>
            <p><strong>Demonstração para Conselho Universitário</strong></p>
        </div>
        """)
        
        gr.Markdown("""
        ### 📋 Como usar:
        1. **Faça upload** de um arquivo de áudio (.mp3, .wav, .m4a)
        2. **Clique em "Processar Áudio"** e aguarde
        3. **Visualize os resultados** nas abas abaixo
        
        > ⚡ O processamento pode levar alguns minutos dependendo do tamanho do arquivo
        """)
        
        # Input de áudio
        with gr.Row():
            audio_input = gr.Audio(
                label="📁 Upload do Arquivo de Áudio",
                type="filepath",
                format="wav"
            )
        
        # Botão de processamento
        process_btn = gr.Button(
            "🚀 Processar Áudio",
            variant="primary",
            size="lg"
        )
        
        # Outputs organizados em abas
        with gr.Tabs():
            
            with gr.TabItem("📊 Status & Estatísticas"):
                status_output = gr.Markdown(label="Status")
                stats_output = gr.Markdown(label="Estatísticas")
            
            with gr.TabItem("🎤 Transcrição"):
                transcription_output = gr.Markdown(label="Transcrição Completa")
            
            with gr.TabItem("📋 Ata Gerada"):
                ata_output = gr.Markdown(label="Ata de Reunião")
        
        # Conectar o botão com a função
        process_btn.click(
            fn=process_audio_file,
            inputs=[audio_input],
            outputs=[status_output, stats_output, transcription_output, ata_output],
            show_progress=True
        )
        
        # Rodapé
        gr.HTML("""
        <div style="text-align: center; margin-top: 30px; padding: 20px; background-color: #f5f5f5; border-radius: 10px;">
            <p><strong>🎓 Sistema desenvolvido para a UFS</strong></p>
            <p>Tecnologias: Whisper AI + PyAnnote + OpenAI GPT + Gradio</p>
            <p><em>Demonstração técnica - TCC Engenharia de Computação</em></p>
        </div>
        """)
    
    return interface

## 🚀 Executar a Aplicação

In [ ]:
# Verificar se tudo está configurado
def check_setup():
    status = []
    
    # Verificar Whisper
    if 'whisper_model' in globals():
        status.append("✅ Whisper: Configurado")
    else:
        status.append("❌ Whisper: Não configurado")
    
    # Verificar OpenAI
    if 'client' in globals():
        status.append("✅ OpenAI: Configurado")
    else:
        status.append("❌ OpenAI: Não configurado")
    
    # Verificar Diarização
    if DIARIZATION_AVAILABLE:
        status.append("✅ Diarização: Disponível")
    else:
        status.append("⚠️ Diarização: Não disponível (sistema funcionará sem separação de speakers)")
    
    print("\n".join(status))
    return all("✅" in s or "⚠️" in s for s in status)

setup_ok = check_setup()

In [ ]:
# Executar a interface
if setup_ok:
    print("🎯 Iniciando Sistema de Geração de Atas - UFS")
    print("📱 A interface será aberta em uma nova aba/janela")
    print("🔗 Ou acesse o link que será exibido abaixo")
    print("\n" + "="*50)
    
    # Criar e executar a interface
    demo = create_interface()
    demo.launch(
        server_name="0.0.0.0",  # Permite acesso externo
        server_port=7860,       # Porta padrão do Gradio
        share=True,             # Criar link público temporário
        debug=False,            # Modo produção
        show_error=True         # Mostrar erros na interface
    )
else:
    print("❌ Sistema não está completamente configurado.")
    print("   Verifique as configurações acima antes de executar.")

## 📝 Instruções de Uso para Demonstração

### Para o Representante da UFS:

1. **Execute todas as células acima** na ordem apresentada
2. **Configure sua chave da OpenAI** na célula de configuração
3. **Execute a última célula** para abrir a interface
4. **Acesse o link** que será gerado (público temporário)

### Recursos da Interface:

- ✅ **Upload simples** - Arraste e solte arquivos de áudio
- ✅ **Processamento visual** - Barra de progresso em tempo real
- ✅ **Resultados organizados** - Abas para cada tipo de informação
- ✅ **Estatísticas detalhadas** - Tempo de fala por participante
- ✅ **Ata profissional** - Formatação adequada para uso oficial

### Formatos de Áudio Suportados:
- `.mp3` - Mais comum
- `.wav` - Melhor qualidade
- `.m4a` - iOS/iPhone

---

**🎯 Esta é uma demonstração técnica do sistema proposto para automatização das atas de reunião do Conselho Universitário da UFS.**